# Download Derived Motion Wind (DMW)

## Para que serve?

Código para fazer o download do produto dos ventos (ABI Derived Motion Wind), de acordo com a data selecionada. 

## Bibliotecas necessárias:

In [ ]:
import boto3                                    # Amazon Web Services (AWS) SDK for Python
from botocore import UNSIGNED                   # boto3 config
from botocore.config import Config              # boto3 config
import os
from datetime import datetime                   # Basic Dates and time types

## Detalhando o código...

Primeiramente, deve-se colocar algumas informações:
- path_dest: diretório em que se deseja salvar o arquivo nc a ser baixado
- yyyymmddhhmn: data no formato 'AnoMesDiaHoraMinuto' (lembrando que são gerados apenas 6 produtos por hora, então nem todos minutos estão disponíveis, verificar no servidor)
- bucket_name: nome do satélite
- product_name: nome do produto conforme está escrito no site do servidor da Amazon, nesse caso é o dos ventos 'ABI-L2-DMWF' 

In [ ]:
path_dest = 'Samples'
yyyymmddhhmn = '202201021000'
bucket_name = 'noaa-goes16'
product_name = 'ABI-L2-DMWF'

Utilizando a biblioteca os, há uma verificação se a pasta de destino já existe, caso não exista ela é criada. Já utilizando a biblioteca datetime, a data é separada na formatação necessária para o download.

In [ ]:
os.makedirs(path_dest, exist_ok=True)

year = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
day_of_year = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%j')
hour = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
min = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')

A seguir, é feito o download do arquivo nc, faz-se uma conexão com o servidor da Amazon Web Services através da biblioteca boto3, e então baixa-se o arquivo buscando-o pelo nome e data escolhida.

In [ ]:
# Initializes the S3 client
s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))

# File structure
prefix = f'{product_name}/{year}/{day_of_year}/{hour}/OR_{product_name}-M6C02_G16_s{year}{day_of_year}{hour}{min}' 

# Seach for the file on the server
s3_result = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter = "/")

# Check if there are files available
if 'Contents' not in s3_result: 
    # There are no files
    print(f'No files found for the date: {yyyymmddhhmn}, Product-{product_name}')
else:
# There are files
    for obj in s3_result['Contents']: 
        key = obj['Key']
        # Print the file name
        file_name = key.split('/')[-1].split('.')[0]

    # Download the file
        if os.path.exists(f'{path_dest}/{file_name}.nc'):
            print(f'File {path_dest}/{file_name}.nc exists')
        else:
            print(f'Downloading file {path_dest}/{file_name}.nc')
            s3_client.download_file(bucket_name, key, f'{path_dest}/{file_name}.nc')